In [6]:
import pandas as pd 
import numpy as np 
import time

In [7]:
# Load months of flight data 
Dec_flight = pd.read_csv('C:\\maha\\508\\rawdata\\ONTIME_REPORTING_12.csv') 

In [8]:
Dec_flight.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 32
0,12,8,7,WN,N8651A,3689,15016,STL,"St. Louis, MO",14679,...,245.0,266.0,1557.0,7,0.0,0.0,18.0,0.0,0.0,NaN
1,12,8,7,WN,N939WN,2600,15016,STL,"St. Louis, MO",14683,...,145.0,125.0,786.0,4,NaN,NaN,NaN,NaN,NaN,NaN
2,12,8,7,WN,N7741C,2770,15016,STL,"St. Louis, MO",14683,...,140.0,131.0,786.0,4,NaN,NaN,NaN,NaN,NaN,NaN
3,12,8,7,WN,N550WN,6654,15016,STL,"St. Louis, MO",14747,...,275.0,256.0,1709.0,7,NaN,NaN,NaN,NaN,NaN,NaN
4,12,8,7,WN,N8319F,3402,15016,STL,"St. Louis, MO",14771,...,270.0,256.0,1735.0,7,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
Dec_flight.isna().sum()

MONTH                       0
DAY_OF_MONTH                0
DAY_OF_WEEK                 0
OP_UNIQUE_CARRIER           0
TAIL_NUM                  457
OP_CARRIER_FL_NUM           0
ORIGIN_AIRPORT_ID           0
ORIGIN                      0
ORIGIN_CITY_NAME            0
DEST_AIRPORT_ID             0
DEST                        0
DEST_CITY_NAME              0
CRS_DEP_TIME                0
DEP_TIME                 5510
DEP_DELAY_NEW            5510
DEP_DEL15                5510
DEP_TIME_BLK                0
CRS_ARR_TIME                0
ARR_TIME                 6045
ARR_DELAY_NEW            7151
ARR_TIME_BLK                0
CANCELLED                   0
CANCELLATION_CODE      619970
CRS_ELAPSED_TIME            0
ACTUAL_ELAPSED_TIME      7151
DISTANCE                    0
DISTANCE_GROUP              0
CARRIER_DELAY          498818
WEATHER_DELAY          498818
NAS_DELAY              498818
SECURITY_DELAY         498818
LATE_AIRCRAFT_DELAY    498818
Unnamed: 32            625763
dtype: int

In [10]:
# Drop observations with missing departure time and tail number.  
Dec_flight.drop(Dec_flight.loc[Dec_flight['DEP_TIME'].isna()].index, axis=0, inplace=True) 
Dec_flight.drop(Dec_flight.loc[Dec_flight['ARR_TIME'].isna()].index, axis=0, inplace=True) 
 
# Drop cancelled flights 
Dec_flight.drop(Dec_flight[Dec_flight['CANCELLED'] == 1].index, inplace = True)

In [11]:
Dec_flight.isna().sum()

MONTH                       0
DAY_OF_MONTH                0
DAY_OF_WEEK                 0
OP_UNIQUE_CARRIER           0
TAIL_NUM                    0
OP_CARRIER_FL_NUM           0
ORIGIN_AIRPORT_ID           0
ORIGIN                      0
ORIGIN_CITY_NAME            0
DEST_AIRPORT_ID             0
DEST                        0
DEST_CITY_NAME              0
CRS_DEP_TIME                0
DEP_TIME                    0
DEP_DELAY_NEW               0
DEP_DEL15                   0
DEP_TIME_BLK                0
CRS_ARR_TIME                0
ARR_TIME                    0
ARR_DELAY_NEW            1106
ARR_TIME_BLK                0
CANCELLED                   0
CANCELLATION_CODE      619718
CRS_ELAPSED_TIME            0
ACTUAL_ELAPSED_TIME      1106
DISTANCE                    0
DISTANCE_GROUP              0
CARRIER_DELAY          492773
WEATHER_DELAY          492773
NAS_DELAY              492773
SECURITY_DELAY         492773
LATE_AIRCRAFT_DELAY    492773
Unnamed: 32            619718
dtype: int

In [12]:
# Investigate the observations with missing Arrival delay information 
MISSING_ARR_DELAY = Dec_flight[Dec_flight['ARR_DELAY_NEW'].isna()] 
 
# Manually calculate the arrival delay value and check if that solves the missing arrival delay information 
MISSING_ARR_DELAY['ARR_DELAY_NEW'] = MISSING_ARR_DELAY['ARR_TIME'] - MISSING_ARR_DELAY['CRS_ARR_TIME'] 
 
# Review the descriptive statistics for the new calculated values to ensure no negatives or errors 
MISSING_ARR_DELAY['ARR_DELAY_NEW'].describe()

<ipython-input-12-c0f7b7da7dd4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MISSING_ARR_DELAY['ARR_DELAY_NEW'] = MISSING_ARR_DELAY['ARR_TIME'] - MISSING_ARR_DELAY['CRS_ARR_TIME']


count    1106.000000
mean      -22.830922
std       786.554676
min     -2305.000000
25%       136.000000
50%       258.000000
75%       375.000000
max      1758.000000
Name: ARR_DELAY_NEW, dtype: float64

In [13]:
#Import aircraft information - tail numbers, manufacture year, and capacity information
aircraft = pd.read_csv('C:\\maha\\508\\rawdata\\B43_AIRCRAFT_INVENTORY.csv', encoding='latin1') 
 
# Drop Duplicates to retain a dictionary 
aircraft.drop_duplicates(subset='TAIL_NUM', inplace=True) 
aircraft.head()

,MANUFACTURE_YEAR,TAIL_NUM,NUMBER_OF_SEATS
0,1944,N54514,0.0
1,1945,N1651M,0.0
2,1953,N100CE,0.0
3,1953,N141FL,0.0
4,1953,N151FL,0.0


In [14]:
# Create dictionary for consistent carrier names using OP_Unique_carrier and Airline_ID 
names = pd.read_csv('C:\\maha\\508\\rawdata\\CARRIER_DECODE.csv') 
 
# Drop Duplicates to retain a dictionary 
names.drop_duplicates(subset='OP_UNIQUE_CARRIER', inplace=True) 
names = names.reset_index(drop=True)
names

,AIRLINE_ID,OP_UNIQUE_CARRIER,CARRIER_NAME
0,21754,2PQ,21 Air LLC
1,20342,Q5,40-Mile Air
2,20342,WRB,40-Mile Air
3,19627,CIQ,A/S Conair
4,19072,AAE,AAA Airlines
...,...,...,...
1739,20379,ZKQ,Zantop International
1740,19771,ZAQ,Zas Airline Of Egypt
1741,21118,37,Zeal 320
1742,22069,ZG,ZIPAIR Tokyo Inc.


In [15]:
#Import Employee information 
employees = pd.read_csv('C:\\maha\\508\\rawdata\\P10_EMPLOYEES.csv')

# Combine Carrier information for different entities and retain entitiy (only domestic), passenger handling (flight attendant), pass_gen_svc_admin (ground service), pilot_copilots and maintanence 
employees = employees[['OP_UNIQUE_CARRIER', 'ENTITY', 'PILOTS_COPILOTS', 'PASSENGER_HANDLING', 'PASS_GEN_SVC_ADMIN', 'MAINTENANCE']] 

# Drop on domestic entities 
employees.drop(employees[employees['ENTITY'] != 'D'].index, inplace = True) 
 
# Combine any remaining duplicates 
employees = employees.groupby('OP_UNIQUE_CARRIER').sum().reset_index() 
 
# Drop Parcel carriers (airlines with no flight attendants) 
employees.drop(employees[employees['PASSENGER_HANDLING'] == 0].index, inplace = True) 
employees = employees.reset_index(drop=True)
employees

,OP_UNIQUE_CARRIER,PILOTS_COPILOTS,PASSENGER_HANDLING,PASS_GEN_SVC_ADMIN,MAINTENANCE
0,AA,8586,8586,15502,9677
1,AS,2893,1062,5737,898
2,B6,2840,4905,3888,726
3,DL,9293,15331,15809,6122
4,F9,1473,2496,154,237
5,G4,953,200,1626,420
6,HA,586,893,1466,419
7,MQ,2109,4923,1510,1565
8,NK,2126,264,3592,395
9,OO,5175,1407,4076,2145


In [16]:
# Import Airport Weather data  
weather_report = pd.read_csv('C:\\maha\\508\\rawdata\\airport_weather_dec_2019.csv') 
weather_report.head()

,STATION,NAME,DATE,AWND,PGTM,PRCP,PSUN,SN32,SNOW,SNWD,...,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT18
0,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,12/1/2019,16.11,NaN,0.04,0.0,0.0,64.0,67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USW00013904,"AUSTIN BERGSTROM INTERNATIONAL AIRPORT, TX US",12/1/2019,10.29,NaN,0.00,0.0,0.0,62.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USW00093721,"BALTIMORE WASHINGTON INTERNATIONAL AIRPORT, MD US",12/1/2019,8.05,NaN,0.62,0.0,0.0,41.0,45.0,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USW00013881,"CHARLOTTE DOUGLAS AIRPORT, NC US",12/1/2019,10.29,NaN,0.60,0.0,0.0,56.0,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USW00093812,"CINCINNATI MUNICIPAL AIRPORT LUNKEN FIELD, OH US",12/1/2019,11.41,NaN,0.09,NaN,NaN,NaN,60.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Import city and airport name dictionary
cities = pd.read_csv('C:\\maha\\508\\rawdata\\airports_list.csv') 
cities.head()

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME,NAME
0,12992,Adams Field,"Little Rock, AR","NORTH LITTLE ROCK AIRPORT, AR US"
1,10257,Albany International,"Albany, NY","ALBANY INTERNATIONAL AIRPORT, NY US"
2,10140,Albuquerque International Sunport,"Albuquerque, NM","ALBUQUERQUE INTERNATIONAL AIRPORT, NM US"
3,10299,Anchorage International,"Anchorage, AK","ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT, A..."
4,10397,Atlanta Municipal,"Atlanta, GA",ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...


In [19]:
# Merge weather report and cities for future merging 
weather_merge = pd.merge(cities, weather_report, how='left', on='NAME') 
weather_merge.head()

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME,NAME,STATION,DATE,AWND,PGTM,PRCP,PSUN,...,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT18
0,12992,Adams Field,"Little Rock, AR","NORTH LITTLE ROCK AIRPORT, AR US",USW00003952,12/1/2019,10.07,NaN,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12992,Adams Field,"Little Rock, AR","NORTH LITTLE ROCK AIRPORT, AR US",USW00003952,12/2/2019,4.03,NaN,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12992,Adams Field,"Little Rock, AR","NORTH LITTLE ROCK AIRPORT, AR US",USW00003952,12/3/2019,4.03,NaN,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12992,Adams Field,"Little Rock, AR","NORTH LITTLE ROCK AIRPORT, AR US",USW00003952,12/4/2019,2.91,NaN,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12992,Adams Field,"Little Rock, AR","NORTH LITTLE ROCK AIRPORT, AR US",USW00003952,12/5/2019,4.92,NaN,0.01,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# limit scope of weather metrics (date, precipitation, snow, max temp, and wind) 
weather = weather_merge[['DATE', 'PRCP', 'SNOW', 'SNWD','TMAX', 'AWND', 'ORIGIN_AIRPORT_ID']]
weather.isna().sum()

DATE                   1
PRCP                   1
SNOW                 621
SNWD                   2
TMAX                   9
AWND                   1
ORIGIN_AIRPORT_ID      0
dtype: int64

In [21]:
# Replace missing snow values with 0 for days with 0 preciptation 
weather.SNOW.where((~weather.SNOW.isna()) & (weather.PRCP != '0.00'), 
                     '0.00', inplace=True) 
 
# Replace missing snow depth (SNWD) values with 0 for days with no snow 
weather.SNWD.where((~weather.SNWD.isna()) & (weather.SNOW != '0.00'), 
                     '0.00', inplace=True)

C:\Users\Maha.Jayapal\Anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


In [22]:
weather.isna().sum()

DATE                 1
PRCP                 1
SNOW                 0
SNWD                 0
TMAX                 9
AWND                 1
ORIGIN_AIRPORT_ID    0
dtype: int64

In [23]:
# Change date format to match flight data (Month & Day_of_Month) 
weather['DATE'] = pd.to_datetime(weather['DATE']) 
weather['MONTH'] = pd.DatetimeIndex(weather['DATE']).month 
weather['DAY_OF_MONTH'] = pd.DatetimeIndex(weather['DATE']).day 
weather

<ipython-input-23-ea21bbfa208e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather['DATE'] = pd.to_datetime(weather['DATE'])
<ipython-input-23-ea21bbfa208e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather['MONTH'] = pd.DatetimeIndex(weather['DATE']).month
<ipython-input-23-ea21bbfa208e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

,DATE,PRCP,SNOW,SNWD,TMAX,AWND,ORIGIN_AIRPORT_ID,MONTH,DAY_OF_MONTH
0,2019-12-01,0.00,0.00,0.00,270.0,10.07,12992,12.0,1.0
1,2019-12-02,0.00,0.00,0.00,40.0,4.03,12992,12.0,2.0
2,2019-12-03,0.00,0.00,0.00,240.0,4.03,12992,12.0,3.0
3,2019-12-04,0.00,0.00,0.00,320.0,2.91,12992,12.0,4.0
4,2019-12-05,0.01,0.00,0.00,150.0,4.92,12992,12.0,5.0
...,...,...,...,...,...,...,...,...,...
2972,2019-12-27,0.00,32.0,35.0,320.0,5.82,10713,12.0,27.0
2973,2019-12-28,0.00,29.0,39.0,300.0,2.24,10713,12.0,28.0
2974,2019-12-29,0.04,31.0,32.0,140.0,6.26,10713,12.0,29.0
2975,2019-12-30,0.00,31.0,34.0,130.0,2.46,10713,12.0,30.0


In [24]:
# Merge aircraft information into flight data file 
Dec_flight = pd.merge(Dec_flight, aircraft, how="left", on='TAIL_NUM')

In [25]:
#Merge names and flight data for consistent airline names 
Dec_flight = pd.merge(Dec_flight, names, how='left', on=['OP_UNIQUE_CARRIER'])

In [26]:
#Merge Employee
Dec_flight = pd.merge(Dec_flight, employees, how='left', on=['OP_UNIQUE_CARRIER'])

In [27]:
Dec_flight.isna().sum()

MONTH                       0
DAY_OF_MONTH                0
DAY_OF_WEEK                 0
OP_UNIQUE_CARRIER           0
TAIL_NUM                    0
OP_CARRIER_FL_NUM           0
ORIGIN_AIRPORT_ID           0
ORIGIN                      0
ORIGIN_CITY_NAME            0
DEST_AIRPORT_ID             0
DEST                        0
DEST_CITY_NAME              0
CRS_DEP_TIME                0
DEP_TIME                    0
DEP_DELAY_NEW               0
DEP_DEL15                   0
DEP_TIME_BLK                0
CRS_ARR_TIME                0
ARR_TIME                    0
ARR_DELAY_NEW            1106
ARR_TIME_BLK                0
CANCELLED                   0
CANCELLATION_CODE      619718
CRS_ELAPSED_TIME            0
ACTUAL_ELAPSED_TIME      1106
DISTANCE                    0
DISTANCE_GROUP              0
CARRIER_DELAY          492773
WEATHER_DELAY          492773
NAS_DELAY              492773
SECURITY_DELAY         492773
LATE_AIRCRAFT_DELAY    492773
Unnamed: 32            619718
MANUFACTUR

In [28]:
#rop airlines with missing employee data 
Dec_flight.drop(Dec_flight.loc[Dec_flight['PASSENGER_HANDLING'].isna()].index, axis=0, inplace=True)

In [29]:
Semi_final = Dec_flight.reset_index(drop=True)
Semi_final.isna().sum()

MONTH                       0
DAY_OF_MONTH                0
DAY_OF_WEEK                 0
OP_UNIQUE_CARRIER           0
TAIL_NUM                    0
OP_CARRIER_FL_NUM           0
ORIGIN_AIRPORT_ID           0
ORIGIN                      0
ORIGIN_CITY_NAME            0
DEST_AIRPORT_ID             0
DEST                        0
DEST_CITY_NAME              0
CRS_DEP_TIME                0
DEP_TIME                    0
DEP_DELAY_NEW               0
DEP_DEL15                   0
DEP_TIME_BLK                0
CRS_ARR_TIME                0
ARR_TIME                    0
ARR_DELAY_NEW             905
ARR_TIME_BLK                0
CANCELLED                   0
CANCELLATION_CODE      543106
CRS_ELAPSED_TIME            0
ACTUAL_ELAPSED_TIME       905
DISTANCE                    0
DISTANCE_GROUP              0
CARRIER_DELAY          433111
WEATHER_DELAY          433111
NAS_DELAY              433111
SECURITY_DELAY         433111
LATE_AIRCRAFT_DELAY    433111
Unnamed: 32            543106
MANUFACTUR

In [30]:
# Dropping Redundant feature 
Final = Semi_final.drop(columns=['ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_NAME', 'DEST_AIRPORT_ID', 'DEST_CITY_NAME'])
 
# Dropping extensive missing features 
Final = Final.drop(columns=['CARRIER_DELAY', 'WEATHER_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'NAS_DELAY', 'CANCELLATION_CODE', 'Unnamed: 32', 'CANCELLED']) 

In [31]:
Final.isna().sum()

MONTH                      0
DAY_OF_MONTH               0
DAY_OF_WEEK                0
OP_UNIQUE_CARRIER          0
TAIL_NUM                   0
OP_CARRIER_FL_NUM          0
ORIGIN                     0
DEST                       0
CRS_DEP_TIME               0
DEP_TIME                   0
DEP_DELAY_NEW              0
DEP_DEL15                  0
DEP_TIME_BLK               0
CRS_ARR_TIME               0
ARR_TIME                   0
ARR_DELAY_NEW            905
ARR_TIME_BLK               0
CRS_ELAPSED_TIME           0
ACTUAL_ELAPSED_TIME      905
DISTANCE                   0
DISTANCE_GROUP             0
MANUFACTURE_YEAR       13074
NUMBER_OF_SEATS        13074
AIRLINE_ID                 0
CARRIER_NAME               0
PILOTS_COPILOTS            0
PASSENGER_HANDLING         0
PASS_GEN_SVC_ADMIN         0
MAINTENANCE                0
dtype: int64